# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4885, uptime 0:00:34
mini_beamline                    RUNNING   pid 4886, uptime 0:00:34
random_walk                      RUNNING   pid 4887, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4888, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4889, uptime 0:00:34
simple                           RUNNING   pid 4890, uptime 0:00:34
thermo_sim                       RUNNING   pid 4891, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4892, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-15 20:15:41
Persistent Unique Scan ID: '47382d03-463e-451d-afb7-4598d0ba5b71'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:15:42.0 |     -1.000 |      95092 |
|         2 | 20:15:42.0 |     -0.500 |      95895 |
|         3 | 20:15:42.0 |      0.000 |      96759 |
|         4 | 20:15:42.0 |      0.500 |      95991 |
|         5 | 20:15:42.1 |      1.000 |      94558 |


+-----------+------------+------------+------------+
generator scan ['47382d03'] (scan num: 3)





('47382d03-463e-451d-afb7-4598d0ba5b71',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/47382d03-463e-451d-afb7-4598d0ba5b71-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-15 20:15:42
Persistent Unique Scan ID: '243f8d62-bbe9-4d81-a0db-87b2ae744ca6'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:15:42.5 |          -1 | 1055562224 |


|         2 | 20:15:42.6 |          -0 | 1053790745 |


|         3 | 20:15:42.7 |           0 | 1053810696 |
|         4 | 20:15:42.8 |           0 | 1055504187 |


|         5 | 20:15:42.9 |           1 | 1057991112 |


+-----------+------------+-------------+------------+
generator scan ['243f8d62'] (scan num: 4)





('243f8d62-bbe9-4d81-a0db-87b2ae744ca6',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-15 20:15:43
Persistent Unique Scan ID: '5cc738a5-965b-4a76-a6ee-7a62b1c0dd43'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:15:43.4 | 1075683816 |


|         2 | 20:15:44.3 | 1128089257 |


|         3 | 20:15:45.3 | 1108775415 |


...

|        28 | 20:16:10.4 | 1055594068 |


|        29 | 20:16:11.4 | 1079430151 |


|        30 | 20:16:12.4 | 1129623052 |


+-----------+------------+------------+
generator count ['5cc738a5'] (scan num: 5)





('5cc738a5-965b-4a76-a6ee-7a62b1c0dd43',)